In [1]:
import pandas as pd
import time
import os

class ldLinkUtilities:
    
    def __init__(self, file_path):
        self.file_path = self
        
    def appendTo2DArrayOrMakeNewArrayIn2DArray (TwoDarray, valueToAppend):
        added = False
        for i in range(len(TwoDarray)):
            if len(TwoDarray[i]) < 1000:
                TwoDarray[i].append(valueToAppend)
                added = True
        if added == False: #meaning array or all arrays were full, make new array and add onto there
            TwoDarray.append([]) #should append to last of list?
            TwoDarray[-1].append(valueToAppend)
            added = True
            
    def hasKey (d, key):
        if key in d:
            return True
        else:
            return False
        
    def numberOfElementsInDict (d):
        count = 0
        for key in d:
            for lst in d[key]: #d[key] is the entire list, within this list is x amount of lists
                count += len(lst)
        #print(count)
                
    def listOfListsToListOfStrings (listToChangeElementsOf):
        
        for i in range(len(listToChangeElementsOf)):
            tempString = ""
            tempString += "\""
            for j in range(len(listToChangeElementsOf[i])): #now at a specific list within the list
                if isinstance(listToChangeElementsOf[i][j], float) == False:
                    if j < len(listToChangeElementsOf[i]) - 1: # if not at the end
                        #print(listToChangeElementsOf[i][j])
                        tempString += "".join((listToChangeElementsOf[i][j],"\\n")) #separator is "\n"
                    else: #if at the end
                        tempString += listToChangeElementsOf[i][j]
            tempString += "\""
            listToChangeElementsOf[i] = tempString #actually replacing the elements for a string
        return listToChangeElementsOf
    
    ###DEPRICATED#####
    def dictToString (d, sep):
        
        result = ""
        for key in d:
            for string in d[key]:
                if string != d[key][-1]: #if not last element
                    result += string + sep #sep should be " " for a unix command
                else:
                    result += string
        return result
    
    def TwoDimArrToString (lst, sep):
        
        result = ""
        for string in lst:
            if string != lst[-1]: #if not last element
                result += string + sep #sep should be " " for a unix command
            else:
                result += string
        return result
    
    ######VERY SLOW########
    def replaceChridForRsid (newD, dfasReference):
        for key in newD:
            start = time.time()
            for lstIdx in range(len(newD[key])): #now at indv chrIDs
                for chrIdIdx in range(len(newD[key][lstIdx])):
                    chrId = newD[key][lstIdx][chrIdIdx]
                    #print(chrId)
                    rsId = dfasReference.loc[dfasReference["chrID"] == chrId]["rsID"] ####CRUCIAL PART-->finds rsID equivalent of chrID
                    #print(rsId.item())
                    newD[key][lstIdx][chrIdIdx] = rsId.item() #the count1 and count2 serve as indices
                    #print(newD[key][lstIdx][chrIdIdx]) 
            end = time.time()
            time_taken = end - start
            print(f'Done replacing variants in {key}, time taken : {time_taken}.')
        return newD
    
    ####NEW ONE FOR ^#######
    def replaceChridIn2DList (lst, dfasReference):
        for row in range(len(lst)): #now at indv chrIDs
            start = time.time()
            for col in range(len(lst[row])):
                chrId = lst[row][col]
                rsId = dfasReference.loc[dfasReference["chrID"] == chrId]["rsID"] ####CRUCIAL PART-->finds rsID equivalent of chrID
                lst[row][col] = rsId.item()
            end = time.time()
            time_taken = end - start
            print(f'Done replacing variants in row {row}, time taken : {time_taken}.')
        return lst
    
    def dictToList (d):
        masterlist = []
        for key,value in d.items():
            for lsts in value:
                masterlist.append(lsts)
        return masterlist
    
    def CSVtoDataFrame (csv):
        df = pd.read_csv(csv)
        return df
    
    def makeChrIDColumn (df):
        df.insert(3,"chrID", "chr" + df["chr"].apply(str) + ":" + df["bp"].apply(str), allow_duplicates=True) 
        
    def chrIDColumnToDictionary (lst,dOfChrIds): #Data structure of dict: {key : [[],[],...[]], key: [[],[],...[]], ... key: [[],[],...[]]}
        for i in range(len(lst)):
            keyName = lst[i].split(":")[0]
            
            if ldLinkUtilities.hasKey(dOfChrIds, keyName) == True: #most likely it will already exist
                ldLinkUtilities.appendTo2DArrayOrMakeNewArrayIn2DArray (dOfChrIds[keyName], lst[i])
            else: #will only happen the first time for each chr, should never be over 1000 bc just started
                dOfChrIds[keyName] = [[]]
                dOfChrIds[keyName][0].append(lst[i])
    
    def makeAndGetFolderNameForBac (ogCSVPathOfBacVariants, parent_dir):
        nameOfBacteria = ""
        splitt = ogCSVPathOfBacVariants.split(".")
        try:
            nameOfBacteria = splitt[1]
            if nameOfBacteria == '':
                nameOfBacteria = splitt[2]
        except IndexError:
            pass
        
        print(nameOfBacteria)
        folderPath = os.path.join(parent_dir, nameOfBacteria)
        try:
            os.mkdir(folderPath)
        except OSError:
            pass
        return nameOfBacteria
        
    def runLDLinkCommandForAListOfInputs(listOfStringUnixInputs, root_dir, folderNameToPutResultsIn):
        count = 1
        
        ##NEW FEATURE: Identify where it left off (at what element in the list of string inputs)
        listOfCurrentOutputFilesInBacteriaFile = []
        print(os.path.join(root_dir, folderNameToPutResultsIn))
        for root, dirs, files in os.walk(os.path.join(root_dir, folderNameToPutResultsIn)):
            for name in files:
                listOfCurrentOutputFilesInBacteriaFile.append(name)
        #print(listOfCurrentOutputFilesInBacteriaFile)
        for i in listOfStringUnixInputs:
            try:
                chrom = i.split("\"")[1].split("\\n")[0].split(":")[0] #both of these try and catach do same thing, just that one (first one) is for the exception when the list of stringinput includes only one  SNP
            except KeyError:
                selectEleToGetChrom = i.split("\\n")[1]
                chrom = selectEleToGetChrom.split(":")[0]
            filename = "%s_%s.csv"%(chrom,count)
            #print(filename)
            if filename in listOfCurrentOutputFilesInBacteriaFile:
                count += 1
                continue
            else:
                print("Processsing %s"%(filename))
                #other parameters that can be changed: pop, r2, maf
                cmd = "curl -k -H \"Content-Type: application/json\" -X POST -d '{\"snps\": %s, \"pop\": \"EUR\", \"r2_threshold\": \"0.3\", \"maf_threshold\": \"0.01\"}' 'https://ldlink.nci.nih.gov/LDlinkRest/snpclip?token=8a70a1b4adf8' >> %s/%s/%s"%(i, root_dir, folderNameToPutResultsIn, filename)
                count += 1
                
                os.system(cmd)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#possibly missing (the files where the program stopped): chr21_154, holdemania chr11_14.csv


In [ ]:
import time
import os

parent_dir = "/content/drive/MyDrive/MiBioGen_filtered"

dir_for_results = "/content/drive/MyDrive/LDLinkOutput"

listOfCSVPathsToProcess = []

for root,dirs,files in os.walk(parent_dir):
    for name in files:
        listOfCSVPathsToProcess.append(os.path.join(root, name))

###THE PIPELINE
count = 0
for i in listOfCSVPathsToProcess: #i is the og bac sum stats file
    count += 1
    print("Processing %s...."%(i))
    print("%s out of %s completed"%(count - 1, len(listOfCSVPathsToProcess)))
    start = time.time()
    
    df = ldLinkUtilities.CSVtoDataFrame(i)
    
    ldLinkUtilities.makeChrIDColumn(df)
    
    #specified column to a list
    listOfChrIds = list(df["chrID"])
    #prepare the dictionary
    dOfChrIds = {}
    
    ldLinkUtilities.chrIDColumnToDictionary(listOfChrIds,dOfChrIds)
    
    listOfStringUnixInputs = ldLinkUtilities.listOfListsToListOfStrings(ldLinkUtilities.dictToList(dOfChrIds))
    
    dir_toPlaceLDLinkOutput = "/content/drive/MyDrive/LDLinkOutput"
    
    folderNameToPutResultsIn = ldLinkUtilities.makeAndGetFolderNameForBac(i, dir_toPlaceLDLinkOutput)
    
    ldLinkUtilities.runLDLinkCommandForAListOfInputs(listOfStringUnixInputs, dir_for_results, folderNameToPutResultsIn)
    
    end = time.time()
    print("Time taken to complete bacteria file was  %s seconds"%(end - start))

Processing /content/drive/MyDrive/MiBioGen_filtered/addedPgenus.Ruminococcus1.id.11373.summary.txt.csv....
0 out of 119 completed
Ruminococcus1
/content/drive/MyDrive/LDLinkOutput/Ruminococcus1
Time taken to complete bacteria file was  6.161752223968506 seconds
Processing /content/drive/MyDrive/MiBioGen_filtered/addedPgenus..Eubacteriumnodatumgroup.id.11297.summary.txt.csv....
1 out of 119 completed
Eubacteriumnodatumgroup
/content/drive/MyDrive/LDLinkOutput/Eubacteriumnodatumgroup
Time taken to complete bacteria file was  5.39015793800354 seconds
Processing /content/drive/MyDrive/MiBioGen_filtered/addedPgenus.RuminococcaceaeUCG003.id.11361.summary.txt.csv....
2 out of 119 completed
RuminococcaceaeUCG003
/content/drive/MyDrive/LDLinkOutput/RuminococcaceaeUCG003
Time taken to complete bacteria file was  5.157476902008057 seconds
Processing /content/drive/MyDrive/MiBioGen_filtered/addedPgenus.Olsenella.id.822.summary.txt.csv....
3 out of 119 completed
Olsenella
/content/drive/MyDrive/LDL

In [ ]:
while True:pass